In [ ]:
import os
import pandas as pd
import numpy as np
import pyreadstat as prs

import config as c

In [ ]:
print(c.db2018)
for f in sorted(os.listdir(c.db2018)):
    print(f)

In [ ]:
stataDirs = [c.db2018, 
             c.db2017, 
             c.db2016, 
             c.db2015, 
             c.db2014, 
             c.db2013, 
             c.db2012, 
             c.db2011, 
             c.db2010, 
             c.db2009]

for stataDir in stataDirs:
    print('\n')
    print(stataDir)
    cols = 0
    savs = 0
    for f in os.listdir(stataDir):
        if 'deadmen' not in f:
            continue
        filePath = os.path.join(stataDir, f)
        print(f)
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print(df.head())
        print('count: {} -- {}'.format(len(df.RegNo.unique()), np.sort(df.RegNo.unique()).astype(int)))
        print('number_rows: {}'.format(meta.number_rows))
        print('column_names, n = {}'.format(len(meta.column_names), meta.column_names))
        print('column_labels, n = {}\n'.format(len(meta.column_labels), meta.column_labels))
        colLabels = meta.column_labels
        for l in colLabels:
            print(l)
    break
    print('total savs = {}'.format(savs))
    print('total cols = {}'.format(cols))

## Compare all files in a single year

In [ ]:
for stataDir in stataDirs:
    n=1
    #print('\n{}'.format(stataDir))
    for f in os.listdir(stataDir):
        #if 'deadmen' not in f:
        #    continue
        filePath = os.path.join(stataDir, f)
        print('#{} {:>5s} {}'.format(n, '', f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print('{}'.format('-'*40))
        #print(df.info())
        #print('{}'.format('-'*40))
        print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))

        col_names = meta.column_names
        col_labels = meta.column_labels
        col_names_labels = dict(zip(col_names, col_labels))
        print('Names: {}  Labels: {}'.format(len(col_names), len(col_labels)))
        i = 1
        for name, label in col_names_labels.items():
            unique_cnt = len(df[name].unique())
            print('{:<3} {:<4} {:<20}  {}'.format(i, unique_cnt, name, label))
            i+=1
    
        n+=1
        #vvl_dict = meta.variable_value_labels
        #for k1, v1 in vvl_dict.items():
        #    print('  {}  (value type: {})'.format(k1, type(v1)))
        #    for k2, v2 in v1.items():
        #        print('    {}  {}'.format(k2, v2))
            
        print('{}\n'.format('-'*80))
    
    break

## Compare a single file across all years

In [ ]:
specific_file = 'sysschedule.sav'
for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    for f in os.listdir(stataDir):
        if specific_file not in f:
            continue
            
        filePath = os.path.join(stataDir, f)
        print('{:<5} {}'.format(year, f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        print(df.head())
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print('{}'.format('-'*40))
        print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))

        col_names = meta.column_names
        col_labels = meta.column_labels
        col_names_labels = dict(zip(col_names, col_labels))
        print('Names: {}  Labels: {}'.format(len(col_names), len(col_labels)))
        i = 1
        for name, label in col_names_labels.items():
            unique_cnt = len(df[name].unique())
            print('{:<3} {:<6} {:<20}  {}'.format(i, unique_cnt, name, label))
            i+=1
        
        print('\n---- Fields with codes, and their lookup values ---')
        vvl_dict = meta.variable_value_labels
        for k1, v1 in vvl_dict.items():
            print('{:<5}'.format(k1, type(v1)))
            for k2, v2 in v1.items():
                print('  {:<5} {}'.format(k2, v2))
            
        print('{}\n'.format('-'*80))
        break

## Check number of unique regions and their values

In [ ]:
specific_file = 'sysschedule.sav'
for stataDir in stataDirs:
    year = os.path.basename(stataDir)[:4]
    for f in os.listdir(stataDir):
        if specific_file not in f:
            continue
            
        filePath = os.path.join(stataDir, f)
        print('{:<5} {}'.format(year, f))
        
        df, meta = prs.read_sav(filePath)#, apply_value_formats=True, formats_as_category=True)
        object_methods = [meta for meta in dir(object) if callable(getattr(object, meta))]
        print('Rows: {}  Cols: {}'.format(df.shape[0], df.shape[1]))
        
        regions = np.sort(df.RegNo.unique()).astype(int)
        print('Count: {} -- Region Values: {}'.format(len(regions), regions))
        print('{}\n'.format('-'*80))
    
    #break